# Kryptologie Lab - Übung 05

In [234]:
sbox = [0xE, 0x4, 0xD, 0x1, 0x2, 0xF, 0xB, 0x8, 0x3, 0xA, 0x6, 0xC, 0x5, 0x9, 0x0, 0x7]
perm = [1, 5, 9, 13, 2, 6, 10, 14, 3, 7, 11, 15, 4, 8, 12, 16]

In [235]:
def sbox_block(value, inverse=False):
    if inverse:
        return sbox.index(value)
    else: 
        return sbox[value]

def executeSbox(value,inverse=False):
    block0 = sbox_block(value & 0xF, inverse)
    block1 = sbox_block((value & 0xF0) >> 4, inverse)
    block2 = sbox_block((value & 0xF00) >> 8, inverse)
    block3 = sbox_block((value & 0xF000) >> 12, inverse)
    return (block3 << 12) | (block2 << 8) | (block1 << 4) | block0

def permutation(value,inverse=False):
    output = 0
    for i in range(1, 16):
        bit = value & 0x1
        output = output | (bit << (perm[i] - 1))
        value = value >> 1
    return output

def addRoundKey(value, key):
    return value ^ key

In [236]:
def encrypt(value, key):
    for i in range(3):
        value = addRoundKey(value, key)
        value = executeSbox(value)
        value = permutation(value)
    value = addRoundKey(value, key)
    value = executeSbox(value)
    value = addRoundKey(value, key)
    return value

In [237]:
for i in range(0, 16):
    encrypted = sbox_block(i)
    decrypted = sbox_block(encrypted, inverse=True)
    assert i == decrypted

value = 0xABCD
encrypted = executeSbox(value)
decrypted = executeSbox(encrypted, inverse=True)
assert value == decrypted

encrypt(0xABCD, 0xFABE)

50256

## Lineare Kryptonanalyse

In [238]:
import random

In [239]:
def getBit(val, position):
    return (val >> position) & 0x1

def expandKey(subkey):
    return ((subkey & 0xF) << 4) ^ ((subkey & 0xF0) << 8)

def getProbabilityOfKeys(pairs):
    possibleKeys = dict()
    # iterate possible keys
    for subkey in range(0xFF+1):
        key = expandKey(subkey)
        hits = 0
        # iterate text-cipher pairs
        for pair in pairs:
            x = pair[0]
            y = pair[1]
            v4 = addRoundKey(y, key)
            u4 = executeSbox(v4, inverse=False)
            approx = getBit(x,4) ^ getBit(x,6) ^ getBit(x,7) ^ getBit(u4,5) ^ getBit(u4,7) ^ getBit(u4,13) ^ getBit(u4,15)
            if approx == 0:
                hits += 1
        possibleKeys[subkey] = hits/len(pairs)
    return possibleKeys

def getBestSubKeyOrder(probabilities):
    for data in probabilities.items():
        probabilities.update({data[0]: abs(data[1] - 0.5)})
    return sorted(probabilities.items(), key=lambda item: item[1], reverse=True)

In [242]:
key = 0xFABE
amount = 1000

M = []
for i in range(amount):
    input = random.getrandbits(16)
    M.append((input, encrypt(input, key)))
probabilities = getProbabilityOfKeys(M)
subkeysInOrderWithProbability = getBestSubKeyOrder(probabilities)
subkeysInOrder = [entry[0] for entry in subkeysInOrderWithProbability]
print(subkeysInOrder)

[119, 146, 203, 49, 202, 54, 156, 97, 154, 210, 72, 155, 34, 55, 158, 178, 199, 123, 129, 98, 115, 140, 194, 198, 39, 130, 141, 191, 201, 221, 6, 42, 43, 71, 187, 208, 209, 231, 18, 99, 114, 137, 147, 207, 250, 53, 62, 118, 131, 157, 188, 186, 22, 35, 38, 135, 136, 206, 12, 70, 237, 251, 10, 25, 29, 50, 51, 92, 124, 134, 180, 195, 255, 48, 96, 11, 86, 100, 103, 111, 120, 149, 151, 200, 204, 214, 66, 139, 232, 19, 23, 75, 132, 153, 183, 205, 211, 253, 17, 68, 79, 80, 113, 238, 240, 0, 28, 47, 101, 108, 110, 121, 159, 160, 185, 217, 235, 248, 106, 122, 128, 164, 175, 218, 219, 1, 56, 57, 65, 102, 126, 127, 171, 179, 196, 212, 213, 220, 243, 244, 32, 33, 58, 59, 67, 69, 77, 78, 90, 172, 226, 241, 20, 24, 36, 40, 133, 189, 245, 252, 8, 14, 15, 26, 91, 168, 173, 31, 46, 74, 83, 109, 184, 190, 197, 234, 3, 73, 166, 169, 193, 227, 236, 239, 21, 45, 52, 60, 63, 82, 89, 93, 94, 104, 116, 117, 125, 143, 150, 170, 216, 7, 16, 76, 81, 107, 112, 144, 162, 165, 167, 174, 176, 192, 228, 229, 30, 37, 

## Brutforce in better order

In [243]:
for key in subkeysInOrder:
    for brutforceSubkey in range(0xFF+1):
        expandedKey = expandKey(key)
        testKey = (brutforceSubkey & 0xF) ^ ((brutforceSubkey & 0xF0) << 12) ^ expandedKey
        print("{0:b}".format(testKey))

111000001110000
111000001110001
111000001110010
111000001110011
111000001110100
111000001110101
111000001110110
111000001110111
111000001111000
111000001111001
111000001111010
111000001111011
111000001111100
111000001111101
111000001111110
111000001111111
10111000001110000
10111000001110001
10111000001110010
10111000001110011
10111000001110100
10111000001110101
10111000001110110
10111000001110111
10111000001111000
10111000001111001
10111000001111010
10111000001111011
10111000001111100
10111000001111101
10111000001111110
10111000001111111
100111000001110000
100111000001110001
100111000001110010
100111000001110011
100111000001110100
100111000001110101
100111000001110110
100111000001110111
100111000001111000
100111000001111001
100111000001111010
100111000001111011
100111000001111100
100111000001111101
100111000001111110
100111000001111111
110111000001110000
110111000001110001
110111000001110010
110111000001110011
110111000001110100
110111000001110101
110111000001110110
110111000001110111
